In [ ]:
import matplotlib
import numpy as np
import seaborn as sns
import pandas as pd
import sklearn as sk
import sys
import matplotlib.pyplot as plt
from jedi.api.refactoring import inline

from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RANSACRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.ensemble import AdaBoostRegressor

from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, precision_recall_curve, roc_curve, roc_auc_score

#MNIST DATASET
# Estructura de 28 X 28 pixeles = 784
#%matplotlib inline
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
mnist = fetch_openml(name = 'mnist_784')


#print(mnist)
#print(len(mnist['data']))

# VISUALIZACION
X, y = mnist['data'], mnist['target']

print(X)
print(y)

y = y.astype('float')       # casting
print(y)

print(X.shape)
print(y.shape)

print(X[69999])
print(y[69999])

def viz(n):
    plt.imshow(X[n].reshape(28, 28))
    plt.show()
    return

#viz(69999)
type(y)


#------------- Localizar el número 4 y graficar la imagen
type(y)

print(y == 4)       # indica los indices que son iguales a 4 con VERDADERO O FALSO
np.where(y == 4)    # regresa los indices que contienen el número 4

print(y[69977])     # indice del arraglo con valor a 4

# a) -- Gráfica con código tradicional
_ = X[69977]
_image = _.reshape(28, 28)
plt.imshow(_image)
#plt.show()

# b) Con la función creada anteriormente
#viz(69977)


# ----------------------------------------------------- Stochastic Gradient Descend (SGD)

# Método 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

# Método 2
num_split = 60000
X_train, X_test, y_train, y_test = X[:num_split], X[num_split:], y[:num_split], y[num_split:]

#---- Shuffling the DataSet
shuffle_index = np.random.permutation(num_split)            # Permutación es la variación del orden o posición de elementos.
X_train, y_train = X_train[shuffle_index], y_train[shuffle_index]

# TRAINING A BINARY CLASSSIFIER

# Necesitamos convertir a cero nuestra variable objetivo
y_train_0 = (y_train == 0)      # Hace todo el arreglo a cero de manera automática. Sin necesidad de utilizar un ciclo
y_test_0 = (y_test == 0)



# Stochastic Gradient Descend

# Entrenamiento
clf = SGDClassifier(random_state= 0)        # Random State indica el punto de partida (el origen)
clf.fit(X_train, y_train_0)

#Prediction
viz(1000)
clf.predict(X[1000].reshape(1, -1))

viz(1001)
clf.predict(X[1001].reshape(1, -1))


# Performance Measures and Stratified K-FOLD

clf = SGDClassifier(random_state=0)         # clf clasificador

skfolds = StratifiedKFold(n_splits=3, random_state= 100)

for train_index, test_index in skfolds.split(X_train, y_train_0):
    clone_clf = clone(clf)
    X_train_fold = X_train[train_index]
    y_train_folds = (y_train_0[train_index])
    X_test_fold = X_train[test_index]
    y_test_fold = (y_train_0[test_index])

    clone_clf.fit(X_train_fold, y_train_folds)
    y_pred = clone_clf.predict(X_test_fold)
    n_correct = sum(y_pred == y_test_fold)
    print("{0:.4f}".format(n_correct / len(y_pred)))


# Forma Alternativa
#K-fold cross-validation separa los datos de entrenamiento en K-folds (envolturas-capas) y luego hacer predicciones y evaluarlas
crossValScore = cross_val_score(clf, X_train, y_train_0, cv=3, scoring='accuracy')
print(crossValScore)



# Confusion Matrix
y_train_pred = cross_val_predict(clf, X_train, y_train_0, cv= 3)
print(y_train_pred)

confusionMatrix = confusion_matrix(y_train_0, y_train_pred)
print(confusionMatrix)

cfMatrix = pd.DataFrame(confusion_matrix(y_train_0, y_train_pred))
print(cfMatrix)

cfMatrixF = pd.DataFrame(confusionMatrix, columns=pd.MultiIndex.from_product([['Prediction'], ['Negative', 'Positive']]),
                         index=pd.MultiIndex.from_product([['Actual'], ['Negative', 'Positive']]))

print(cfMatrixF)
# Precision. Proporción de los casos positivos bien clasificados del modelo respecto al total de predicciones positivas.
precisionScore = precision_score(y_train_0, y_train_pred)
print(precisionScore)

# Sensibilidad. Indice de Verdaderos Positivos (TPR). Proporción de los casos positivos bien clasificados del modelo respecto al total real de positivos.
RecallScore = recall_score(y_train_0, y_train_pred)
print(RecallScore)

F1Score = f1_score(y_train_0, y_train_pred)
print(F1Score)


# -------------------------------------------------- Precision - Recall Tradeoff
np.random.seed(0)
clf = SGDClassifier(random_state=0)
clf.fit(X_train, y_train_0)

y_scores = clf.decision_function(X[1000].reshape(1, -1))
print(y_scores)

y_scores = clf.decision_function(X[1001].reshape(1, -1))
print(y_scores)

threshold = 0
y_some_digits_pred = (y_scores > threshold)
print(y_some_digits_pred)

threshold = 40000
y_some_digits_pred = (y_scores > threshold)
print(y_some_digits_pred)

y_scores = cross_val_predict(clf, X_train, y_train_0, cv= 3, method='decision_function')

plt.figure(figsize=(12, 8))
plt.hist(y_scores, bins=100)
plt.show()

precisions, recalls, threshold = precision_recall_curve(y_train_0, y_scores)

def plotRecallPrecisionThreshold(precision, recall, threshold):
    plt.plot(threshold, precision[:-1], 'b--', label='Precision')
    plt.plot(threshold, recall[:-1], 'g--', label= 'Recall')
    plt.xlabel('Threshold')
    plt.legend(loc='upper left')
    plt.ylim([-0.5, 1.5])

plt.figure(figsize=(12,8))
plotRecallPrecisionThreshold(precisions, recalls, threshold)
plt.show()


#--------------------------------------------- Alterando la Curva Precision-Recall --------------------------------------
plt.figure(figsize= (12,8))
plt.plot(precisions, recalls)
plt.xlabel('recalls')
plt.ylabel('precisions')
plt.title('PR Curve: precisions/recalls trade off')

#Setting precisions at 90%
print(len(precisions))
print(len(threshold))
plt.figure(figsize=(12,8))
plt.plot(threshold, precisions[1:])

idx = len(precisions[precisions < 0.9])
threshold[idx]

y_train_pred_90 = (y_scores > 21454)

precision90 = precision_score(y_train_0, y_train_pred_90)
recall90 = recall_score(y_train_0, y_train_pred_90)

print(precision90)
print(recall90)

# Setting High Recall Recall Score > 0.9

idx = len(recalls[recalls > 0.9])
threshold[idx]
y_train_pred_90 = (y_scores > threshold[idx])

precision_score(y_train_0, y_train_pred_90)
recall_score(y_train_0, y_train_pred_90)


fpr, tpr, threshold = roc_curve(y_train_0, y_scores)

def plotROCCurve(fpr, tpr, label=None) :
    plt.plot(fpr, tpr, linewidth = 2, label=label)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.axis([0, 1, 0, 1])
    plt.xlabel('FPR')
    plt.ylabel('TPR')
    plt.title('ROC CURVE')

plt.figure(figsize= (12,8))
plotROCCurve(fpr, tpr)
plt.show()

rocAucScore = roc_auc_score(y_train_0, y_scores)
print(rocAucScore)

# Random Forest

f_clf = RandomForestClassifier(random_state=0, n_estimators=100)
y_probas_forest = cross_val_predict(f_clf, X_train, y_train_0, cv= 3, method= 'predict_proba')

y_scores_forest = y_probas_forest[:, 1]
fpr_forest, tpr_forest, threshold_forest = roc_curve(y_train_0, y_scores_forest)

plt.figure(figsize=(12, 8))
plt.plot(fpr, tpr, "b:", label= "SGD")
plotROCCurve(fpr_forest, tpr_forest, 'Random Forest')
plt.legend(loc= 'lower right')
plt.show()

roc_auc_score(y_train_0, y_scores_forest)
f_clf.fit(X_train, y_train_0)

y_train_rf= cross_val_predict(f_clf, X_train, y_train_0, cv=3)
precision_score(y_train_0, y_train_rf)

precision_score(y_train_0, y_train_rf)
recall_score(y_train_0, y_train_rf)

confusion_matrix(y_train_0, y_train_rf)